In [1]:
import tensorflow as tf
from tensorflow import keras as K
from tensorflow.keras import layers

2023-09-09 11:47:03.567369: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# definimos un modelo secuencial

from tensorflow.keras.optimizers import SGD

sgd = SGD(0.01)

model = K.Sequential(
    [
        layers.Dense(2, input_shape=(2,), activation='relu', name='dense1'),
        layers.Dense(1, activation='sigmoid', name='dense2')
    ]
)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense1 (Dense)              (None, 2)                 6         
                                                                 
 dense2 (Dense)              (None, 1)                 3         
                                                                 
Total params: 9 (36.00 Byte)
Trainable params: 9 (36.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
import pandas as pd

data = pd.read_csv('../data/datasets/simple_binary.csv')

data.head()

,Feature_1,Feature_2,Output
0,2.343876,-2.925000,1
1,3.899890,-2.933112,1
2,2.905043,-2.700764,1
3,2.703007,-3.003042,1
4,2.503735,-3.004376,1


In [5]:
y = data.pop('Output')

In [6]:
y

0      1
1      1
2      1
3      1
4      1
      ..
494    1
495    0
496    0
497    1
498    1
Name: Output, Length: 499, dtype: int64

In [7]:
tf_dataset = tf.data.Dataset.from_tensor_slices((data.values, y.values))

In [8]:
for features, targets in tf_dataset.take(5):
    print('Features: {}, Target: {}'.format(features, targets))

Features: [ 2.34387608 -2.92500003], Target: 1
Features: [ 3.89989038 -2.93311163], Target: 1
Features: [ 2.90504265 -2.70076436], Target: 1
Features: [ 2.70300704 -3.00304232], Target: 1
Features: [ 2.50373548 -3.00437636], Target: 1


In [9]:
batch_size = 16
dataset = tf_dataset.batch(batch_size)
dataset = dataset.shuffle(buffer_size=len(data))
train_dataset = dataset.take(int(len(data) * 0.7))
test_dataset = dataset.skip(int(len(data)*0.7))

In [10]:
history = model.fit(train_dataset, epochs=50)

Epoch 1/50
32/32 [==============================] - 3s 8ms/step - loss: 0.6517 - accuracy: 0.4529
Epoch 2/50
32/32 [==============================] - 0s 6ms/step - loss: 0.6161 - accuracy: 0.4770
Epoch 3/50
32/32 [==============================] - 0s 7ms/step - loss: 0.5803 - accuracy: 0.5170
Epoch 4/50
32/32 [==============================] - 0s 6ms/step - loss: 0.5460 - accuracy: 0.5792
Epoch 5/50
32/32 [==============================] - 0s 7ms/step - loss: 0.5124 - accuracy: 0.6493
Epoch 6/50
32/32 [==============================] - 0s 7ms/step - loss: 0.4788 - accuracy: 0.7415
Epoch 7/50
32/32 [==============================] - 0s 8ms/step - loss: 0.4473 - accuracy: 0.8477
Epoch 8/50
32/32 [==============================] - 0s 8ms/step - loss: 0.4180 - accuracy: 0.9178
Epoch 9/50
32/32 [==============================] - 0s 6ms/step - loss: 0.3950 - accuracy: 0.9579
Epoch 10/50
32/32 [==============================] - 0s 6ms/step - loss: 0.3755 - accuracy: 0.9800
Epoch 11/50
32/32 [

In [12]:
# evaluate the network
model.predict([
    [2.7030070442541745,-3.003042320746851],
    [2.5037354760912955,-3.0043763630080838],
    [5.210247695170851,-3.6166396693348912],
    [3.684823874500105,2.36230835847513],
    [2.009693169089723,3.7860940191103807],
    [4.604840132743344,1.405890402381907]
])

1/1 [==============================] - 0s 285ms/step


array([[0.7975878 ],
       [0.7975878 ],
       [0.7404339 ],
       [0.01083415],
       [0.00869661],
       [0.01488256]], dtype=float32)

In [20]:
for i, layer in enumerate(model.layers):
    print(layer.name, layer)
    print(model.layers[i].weights)

dense1 <keras.src.layers.core.dense.Dense object at 0x7fc5a16d6560>
[<tf.Variable 'dense1/kernel:0' shape=(2, 2) dtype=float32, numpy=
array([[-1.3566418,  0.8290571],
       [ 0.5052472,  1.1291589]], dtype=float32)>, <tf.Variable 'dense1/bias:0' shape=(2,) dtype=float32, numpy=array([-0.24589023,  0.08282296], dtype=float32)>]
dense2 <keras.src.layers.core.dense.Dense object at 0x7fc5a16d7700>
[<tf.Variable 'dense2/kernel:0' shape=(2, 1) dtype=float32, numpy=
array([[ 0.5337429],
       [-1.0138276]], dtype=float32)>, <tf.Variable 'dense2/bias:0' shape=(1,) dtype=float32, numpy=array([1.3712858], dtype=float32)>]


In [14]:
print(model.layers[0].weights)
print(model.layers[0].bias.numpy())
print(model.layers[0].bias_initializer)

[<tf.Variable 'dense1/kernel:0' shape=(2, 2) dtype=float32, numpy=
array([[-1.3566418,  0.8290571],
       [ 0.5052472,  1.1291589]], dtype=float32)>, <tf.Variable 'dense1/bias:0' shape=(2,) dtype=float32, numpy=array([-0.24589023,  0.08282296], dtype=float32)>]
[-0.24589023  0.08282296]
